<a href="https://colab.research.google.com/github/raheelam98/langgraph_guru/blob/main/rag_from_scratch/rag_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Leaderboard

1.   [Arena](https://lmarena.ai/)
2.   [Berkeley Function-Calling Leaderboard](https://gorilla.cs.berkeley.edu/leaderboard.html)
3. sora video

4.   [huggingface](https://huggingface.co/spaces/ArtificialAnalysis/Video-Generation-Arena-Leaderboard)



[Making it easier to build human-in-the-loop agents with interrupt](https://blog.langchain.dev/making-it-easier-to-build-human-in-the-loop-agents-with-interrupt/)

### RAG From Scratch

[RAG From Scratch - videos](https://www.youtube.com/watch?v=wd7TZ4w1mSw&list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x)

[rag-from-scratch - github
](https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_1_to_4.ipynb)

**Basic**

* Indexting
* Retrieval
* Generation

**Advance**

* Query transformation
* Routing
* Query contruction
* Indexing
* Retrieval
* Grenration




**Google Generative AI Embeddings**

In [2]:
# Install the required packages:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain-google-genai

In [3]:
# Install the required packages:
%%capture --no-stderr
%pip install langchain_community tiktoken langchainhub chromadb langchain


In [4]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "rag_ai_agent"

In [5]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [6]:
# Initialize the ChatGoogleGenerativeAI with the Gemini model

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model to use
    max_retries=2,
    api_key=gemini_api_key    # Provide the Google API key for authentication
)

In [ ]:
# Invoke the LLM with a query
result = llm.invoke("hi")
result

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-95a66a05-1b9a-46fc-ac9e-604d12249c6b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_core.vectorstores import InMemoryVectorStore

# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001" , # Specify the embedding model to use
#     api_key=gemini_api_key
# )#vector = embeddings.embed_query("hello, world!")
# # vector[:5]

# embeddings.embed_query("Hello")


### **Embedding**

[Vector embeddings](https://platform.openai.com/docs/guides/embeddings#what-are-embeddings)

```
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001" ,
    google_api_key=gemini_api_key  # correct
    )

vector = embeddings.embed_query("hello, world!")
vector[:5]    
```

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=gemini_api_key)

vector = embeddings.embed_query("hello, world!")
vector[:5]

# embeddings.embed_query("Hello")


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001" ,
    google_api_key=gemini_api_key  # correct
    )

vector = embeddings.embed_query("hello, world!")
vector[:10]


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966,
 -0.0018945280462503433,
 0.028477225452661514,
 -0.007562300190329552,
 0.011064725928008556,
 -0.005353901535272598]

#### Indexting - Part 2

In [8]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

**How many tokens a string has before I embed it**

**Count tokens with Tiktoken**

[How to count tokens with tiktoken
](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)



In [17]:
# How many tokens a string has before I embed it
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")


6

**Text embedding models**

In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embd = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001" ,
    google_api_key=gemini_api_key  # correct
    )

query_result = embd.embed_query(question)
document_result = embd.embed_query(document)

len(query_result)

#print(type(query_result))

768

**Cosine similarity is reccomended (1 indicates identical) for OpenAI embeddings.**

In [18]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.8535652119095083


**Document Loaders**

[Document Loaders](https://www.youtube.com/watch?v=wd7TZ4w1mSw&list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x)

In [25]:
import os

# Set a custom User-Agent string
os.environ["USER_AGENT"] = "WebBaseLoaderBot/1.0 (https://lilianweng.github.io/posts/2023-06-23-agent/)"

In [23]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [27]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [33]:
# # Index

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings())

retriever = vectorstore.as_retriever()


ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
model
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing